In [1]:
import torch
!pip install ultralytics
import torchvision.transforms as transforms
import ultralytics
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from torchvision import models
from torch import nn
import os
import glob
from PIL import Image
import cv2
import numpy as np

In [2]:
import ultralytics 
ultralytics.__version__

'8.3.75'

In [3]:
import torch 
torch.__version__

'2.5.1+cu124'

In [4]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4050 Laptop GPU'

In [9]:
import ultralytics 

# Define paths
KITTI_PATH = "C:/Users/User/Desktop/MIXED/obj_detection/image_2"
YOLO_LABELS_PATH = "C:/Users/User/Downloads/labels-7 class/labels"
DATA_PATH = "E:/KITTI Dataset/datapath-7classes-yolo12"

# Create dataset directory structure
os.makedirs(f"{DATA_PATH}/images/train", exist_ok=True)
os.makedirs(f"{DATA_PATH}/images/val", exist_ok=True)
os.makedirs(f"{DATA_PATH}/labels/train", exist_ok=True)
os.makedirs(f"{DATA_PATH}/labels/val", exist_ok=True)

# Get image and label files
image_files = sorted([f for f in os.listdir(KITTI_PATH) if f.endswith(".png")])
label_files = sorted([f for f in os.listdir(YOLO_LABELS_PATH) if f.endswith(".txt")])

# Ensure matching images and labels
image_files = [f for f in image_files if f.replace(".png", ".txt") in label_files]

# Split into train and validation sets
train_images, val_images = train_test_split(image_files, test_size=0.2, random_state=42)

In [11]:
# Function to copy files (ensures both images and labels exist)
def copy_files(files, src_img, src_lbl, dst_img, dst_lbl):
    for file in files:
        img_src = os.path.join(src_img, file)
        lbl_src = os.path.join(src_lbl, file.replace(".png", ".txt"))
        img_dst = os.path.join(dst_img, file)
        lbl_dst = os.path.join(dst_lbl, file.replace(".png", ".txt"))
        
        if os.path.exists(lbl_src):  # Copy only if label exists
            shutil.copy(img_src, img_dst)
            shutil.copy(lbl_src, lbl_dst)

# Copy training and validation files
copy_files(train_images, KITTI_PATH, YOLO_LABELS_PATH, f"{DATA_PATH}/images/train", f"{DATA_PATH}/labels/train")
copy_files(val_images, KITTI_PATH, YOLO_LABELS_PATH, f"{DATA_PATH}/images/val", f"{DATA_PATH}/labels/val")

# ✅ Validation: Check for copied files
train_img_count = len(os.listdir(f"{DATA_PATH}/images/train"))
val_img_count = len(os.listdir(f"{DATA_PATH}/images/val"))

print(f"Training images: {train_img_count}")
print(f"Validation images: {val_img_count}")

# Raise an error if validation folder is empty
if val_img_count == 0:
    raise ValueError("Validation folder is empty. Check dataset splitting and file copying.")


Training images: 5984
Validation images: 1497


In [13]:
from ultralytics import YOLO

# Create dataset YAML file
dataset_yaml = f"""
path: {os.path.abspath(DATA_PATH)}
train: {os.path.abspath(DATA_PATH)}/images/train
val: {os.path.abspath(DATA_PATH)}/images/val
names:
  0: car
  1: pedestrian
  2: van
  3: Cyclist
  4: truck
  5: misc
  6: tram
  7: person_sitting
"""
with open(f"{DATA_PATH}/data.yaml", "w") as f:
    f.write(dataset_yaml)

# Train YOLOv11 model
model = YOLO("yolov8n.pt")
model.train(data=f"{DATA_PATH}/data.yaml", epochs=100, imgsz=640, batch=16 , val=True, verbose=True)

New https://pypi.org/project/ultralytics/8.3.106 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.75  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=E:/KITTI Dataset/datapath-7classes-yolo12/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train15, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

train: Scanning E:\KITTI Dataset\datapath-7classes-yolo12\labels\train... 5984 images, 0 backgrounds, 0 corrupt: 100%|█


train: New cache created: E:\KITTI Dataset\datapath-7classes-yolo12\labels\train.cache


val: Scanning E:\KITTI Dataset\datapath-7classes-yolo12\labels\val... 1497 images, 0 backgrounds, 0 corrupt: 100%|█████

val: New cache created: E:\KITTI Dataset\datapath-7classes-yolo12\labels\val.cache


...

                   all       1497       8231      0.894      0.827      0.902      0.674



100 epochs completed in 3.879 hours.
Optimizer stripped from runs\detect\train15\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train15\weights\best.pt, 6.2MB

Validating runs\detect\train15\weights\best.pt...
Ultralytics 8.3.75  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:34


                   all       1497       8231       0.92      0.819      0.908      0.678
                   car       1338       5889      0.944      0.906      0.965      0.799
            pedestrian        359        916       0.94      0.664      0.823      0.487
                   van        427        570      0.937      0.911      0.959      0.778
               Cyclist        222        308      0.953      0.786      0.886      0.599
                 truck        220        227       0.96      0.952      0.985      0.844
                  misc        156        187      0.915      0.802      0.909      0.696
                  tram         62         95      0.934      0.894      0.966       0.74
        person_sitting         20         39       0.78      0.641      0.768      0.478
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs\detect\train15


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C5A29B8CE0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [15]:
metrics = model.val()

# Print precision, recall, mAP@50, and mAP@50-95
print(f"Precision: {metrics.box.mp:.4f}")  # Mean precision
print(f"Recall: {metrics.box.mr:.4f}")  # Mean recall
print(f"mAP@50: {metrics.box.map50:.4f}")  # Mean AP at IoU 0.5
print(f"mAP@50-95: {metrics.box.map:.4f}")  # Mean AP from 0.5 to 0.95

Ultralytics 8.3.75  Python-3.12.4 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 168 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning E:\KITTI Dataset\datapath-7classes-yolo12\labels\val.cache... 1497 images, 0 backgrounds, 0 corrupt: 100%
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [00:35


                   all       1497       8231      0.921      0.819      0.907      0.678
                   car       1338       5889      0.944      0.905      0.965      0.799
            pedestrian        359        916      0.943      0.664      0.823      0.488
                   van        427        570      0.937      0.912       0.96       0.78
               Cyclist        222        308      0.953      0.783      0.883      0.599
                 truck        220        227      0.961      0.952      0.985      0.843
                  misc        156        187      0.915      0.803      0.909      0.699
                  tram         62         95      0.934      0.894      0.965      0.738
        person_sitting         20         39       0.78      0.641      0.766      0.479
Speed: 0.3ms preprocess, 3.4ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs\detect\train152
Precision: 0.9207
Recall: 0.8193
mAP@50: 0.9070
mAP@50-95: 0.6783


In [17]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load trained YOLOv8 model
model = YOLO("runs/detect/train/weights/best.pt")  # Change path if needed

# Load and run inference on the test image
image_path = "/kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000052.png"  # Change to your image path
results = model(image_path)

# Display results
for r in results:
    img_with_boxes = r.plot()  # Draw detected objects on the image

# Convert BGR to RGB for correct display in Matplotlib
img_rgb = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 6))
plt.imshow(img_rgb)
plt.axis("off")  # Hide axes
plt.show()


FileNotFoundError: /kaggle/input/kitti-dataset/data_object_image_2/training/image_2/000052.png does not exist